In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/IR Project/squadv2')

In [0]:
!ls

bidaf  setup.ipynb


In [0]:
!ls

accuracy_metric.py  evaluate-v2.0.py  loss_function.py	  preprocess.py
batch_generator.py  generate.py       magnitude.py	  __pycache__
data_generator.py   __init__.py       multi_gpu_model.py


In [0]:
!pip install pymagnitude allennlp

In [0]:
from bidaf.models import BidirectionalAttentionFlow
from bidaf.scripts import load_data_generators
from bidaf.scripts import data_download_and_preprocess

data_download_and_preprocess()

emdim = 400
bidaf = BidirectionalAttentionFlow(emdim=emdim, num_highway_layers=2,
                                   num_decoders=1, encoder_dropout=0.4, decoder_dropout=0.6)
train_generator, validation_generator = load_data_generators(batch_size=16, emdim=emdim, shuffle=True)
model = bidaf.train_model(train_generator, epochs=70, validation_generator=validation_generator,
                          save_history=True, save_model_per_epoch=True)

In [0]:
os.chdir('bidaf/scripts/')

In [0]:
!python generate.py ../data/data/dev.csv ../data/data/single.json


In [0]:
!python evaluate-v2.0.py ../data/data/dev-v2.0.json ../data/data/single.json

{
  "exact": 34.38052724669418,
  "f1": 39.30010884313755,
  "total": 11873,
  "HasAns_exact": 61.859649122807014,
  "HasAns_f1": 72.7129204275594,
  "HasAns_total": 5928,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 5945
}


In [0]:
import pandas as pd

test=pd.read_csv('bidaf/data/data/dev.csv')

In [0]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("bidaf/data/data/bidaf-model-2017.09.15-charpad.tar.gz")
i=0
single={}
for index,row in test.iterrows():
  ans=predictor.predict(passage=row['contexts'],question=row['questions'])
  single[row['qids']]=ans['best_span_str']
  print(i,"Done!", end="\r")

In [0]:
import json
with open("bidaf/data/data/single.json", "w") as write_file:
    json.dump(single, write_file)
print("saved results to data/data/single.json")